In [ ]:
#| default_exp utils.data_extractor

In [ ]:
#| hide

from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

# Data Extractor
> Extract games and its features from multiple DB collections.

In [ ]:
#| export

import pandas as pd
import datetime
import numpy as np
import json
from betting_env.datastructure.game_features import *
from betting_env.datastructure.odds import *
from betting_env.datastructure.team_lineup import * 
from betting_env.config.mongo import mongo_init
from typing import Tuple

## Aggregate Data


We provide a function that seeks to retrieve the list of games recorded in our `gameFeatures` MongoDb Collection and aggregate it with its additional features such as `Lineups` information (lineups Collection) and 1x2 and Asian Handicap `odds`(Odds collection).

In [ ]:
# | export


def data_aggregator(
    db_hosts: dict,  # All DB hosts.
    config: dict,  # Database config.
    db_host: str = "prod_atlas",  # Database host name.
    limit: int = None,  # Number of rows to extract.
) -> pd.DataFrame:  # Mapped games.
    "Returns and aggregates games information from multiple Db collections."

    def _odds(
        game_id: str,  # Real-analytics game identifier.
        game_date: datetime.datetime,  # Find the lastest data document prior to `date`.
        market_type: str,  # Type of market required; should one of 1x2 and Asian Handicap.
    ) -> np.ndarray:  # Odds values.
        "Returns game Odds. It can be 1x2 or Asian Handicap."

        if market_type == "1x2":
            return MarketOdds.get_odds_features(
                ra_game_id=game_id, market=market_type, date=game_date
            )[["odds1", "oddsX", "odds2"]].values[0]
        else:
            return MarketOdds.get_latest(
                ra_game_id=game_id, market=market_type, date=game_date
            )[["odds1", "odds2", "line_id"]].values[0]

    def _team_features(
        team_id: str,  # Real-analytics game identifier.
        game_date: datetime.datetime,  # Find the lastest data document prior to `date`.
    ) -> Tuple:  # Lineup values (name:position, ids, slots, formation name, timestamp).
        "Returns lineup features of a given team."

        # Lineup features.
        team_features = TeamSheet.get_latest(ra_team_id=team_id, date=game_date)
        # Team name.
        team_name = team_features.name
        # Players and positions.
        team_lineups_names = json.dumps(
            {player.name: player.position for player in team_features.starting}
        )
        # Players ids.
        team_lienups_ids = list(player.opta_id for player in team_features.starting)
        # Players slots.
        team_lienups_slots = list(player.slot for player in team_features.starting)
        # Formation name.
        formation_name = team_features.starting.first().formation
        # Lineup timestamp.
        lineup_time_stamp = team_features.received_at

        return (
            team_name,
            team_lineups_names,
            team_lienups_ids,
            team_lienups_slots,
            formation_name,
            lineup_time_stamp,
        )

    # Connect to database.
    mongo_init(db_hosts=db_hosts, config=config, db_host=db_host)

    # Extract games.
    games = GameFeatures.get_all_games(limit=limit)
    games = pd.DataFrame(games.as_pymongo())

    # Filter Data.
    games = games[
        [
            "gameId",
            "game_optaId",
            "gameDate",
            "homeTeamId",
            "homeTeam_optaId",
            "awayTeamId",
            "awayTeam_optaId",
            "tgt_gd",
            "tgt_outcome",
        ]
    ]

    # Add 1X2 odds.
    games[["preGameOdds1", "preGameOddsX", "preGameOdds2"]] = games.apply(
        lambda row: _odds(
            game_id=row["gameId"],
            game_date=row["gameDate"],
            market_type="1x2",
        ),
        axis="columns",
        result_type="expand",
    )

    # Add Asian handicap odds.
    games[["preGameAhHome", "preGameAhAway", "LineId"]] = games.apply(
        lambda row: _odds(
            game_id=row["gameId"],
            game_date=row["gameDate"],
            market_type="asian",
        ),
        axis="columns",
        result_type="expand",
    )

    # Add Home team lineup features.
    games[
        [
            "homeTeamName",
            "homeTeamLineup",
            "homeTeamLineupIds",
            "homeTeamLineupSlots",
            "homeTeamFormation",
            "home_team_lineup_received_at",
        ]
    ] = games.apply(
        lambda row: _team_features(
            team_id=row["homeTeamId"], game_date=row["gameDate"]
        ),
        axis="columns",
        result_type="expand",
    )

    # Add away team lineup features.
    games[
        [
            "awayTeamName",
            "awayTeamLineup",
            "awayTeamLineupIds",
            "awayTeamLineupSlots",
            "awayTeamFormation",
            "away_team_lineup_received_at",
        ]
    ] = games.apply(
        lambda row: _team_features(
            team_id=row["awayTeamId"], game_date=row["gameDate"]
        ),
        axis="columns",
        result_type="expand",
    )

    # Map results {homewin -> 0 , draw -> 1, awaywin -> 2}.
    games["tgt_outcome"] = games["tgt_outcome"].map({1.0: 0.0, 0.0: 2.0, 0.5: 1.0})

    return games

In [ ]:
#| include: false

from betting_env.config.localconfig import CONFIG, DB_HOSTS

In [ ]:
# | include: false

data_aggregator(
    db_hosts=DB_HOSTS,
    config=CONFIG,
    db_host= "prod_atlas",
    limit=3,
)

,gameId,game_optaId,gameDate,homeTeamId,homeTeam_optaId,awayTeamId,awayTeam_optaId,tgt_gd,tgt_outcome,preGameOdds1,...,homeTeamLineupIds,homeTeamLineupSlots,homeTeamFormation,home_team_lineup_received_at,awayTeamName,awayTeamLineup,awayTeamLineupIds,awayTeamLineupSlots,awayTeamFormation,away_team_lineup_received_at
0,174dba7291174b4dbbfa9ea12dd944bb45bdd8ed905524...,990997,2018-08-22 18:45:00,126905d14981e6b97912ad4fec354035ccef26cb8ec4e1...,7,419088133137a53bfdb1b7e2e682d223d33a6fa075bbfe...,94,0,1.0,2.62,...,"[12150, 59115, 122806, 54764, 49773, 37339, 43...","[4, 6, 8, 9, 11, 7, 5, 2, 3, 1, 10]",4-4-1-1,2018-08-22 18:15:00,Brentford,"{""Daniel Bentley"": ""GK"", ""Ezri Konsa"": ""DCR"", ...","[79602, 199798, 115382, 114275, 176442, 223911...","[1, 5, 9, 8, 3, 6, 7, 10, 4, 11, 2]",4-2-3-1,2018-08-22 18:15:00
1,219ef70c0e8a803ec1efdb793443edfaa32398690c7829...,991003,2018-08-22 18:45:00,aeb2f56fcedbcf4cd5c780179766996c7bf0b308064541...,5,f8daf96ad35eebf1c0a5886c72734ba7dec366d6637052...,108,0,1.0,1.98,...,"[156685, 15398, 154561, 165183, 61602, 167802,...","[11, 9, 1, 3, 4, 5, 6, 2, 10, 8, 7]",4-2-3-1,2018-08-22 18:15:00,Reading,"{""Leandro Bacuna"": ""DMR"", ""Andy Yiadom"": ""DR"",...","[74297, 87408, 154133, 86857, 17506, 112133, 2...","[4, 2, 8, 6, 5, 3, 1, 7, 10, 11, 9]",4-4-2,2018-08-22 18:15:00
2,0655e244d8d596b5572e86426e2a7ca6178044efa59437...,991013,2018-08-25 14:00:00,9ee012a80cade2df55b71580bf5e238bcd6be6f696fdc1...,45,38ca605bcd29a5a37697ca66e533ae817ced71b6bf275c...,2,-3,2.0,2.62,...,"[52980, 84384, 108799, 83428, 57127, 87396, 20...","[2, 6, 8, 5, 7, 10, 1, 11, 3, 9, 4]",4-2-3-1,2018-08-25 13:30:00,Leeds United,"{""Mateusz Klich"": ""MCR"", ""Luke Ayling"": ""DR"", ...","[72222, 66588, 98760, 155405, 220037, 61810, 8...","[8, 2, 9, 4, 1, 5, 10, 7, 3, 11, 6]",4-1-4-1,2018-08-25 13:30:00


In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()